<h1 style="display: flex; align-items: center; justify-content: center; font-weight: bold; font-size: 2em;">
  <img src="/Users/jasonjoelpinto/Documents/GitHub/python-datascience-projects/016. sentiment_analysis_NLP_project/dataset/icons8-twitter-pastel/icons8-twitter-256.png" alt="Your Image" style="height: 2.5em; margin-right: 15px;">
  Twitter Post Classification
</h1>


## 👋🏼 **Introduction**

<p style="text-align: justify;">I am a data scientist. And my company wants me to build and NLP pipeline where it wants me to categorize the tweets as racist/sexist or not. The problem that currently we are facing is that there are many people on twitter which is now X, where they do now have someone to talk to when they get suicidical taughts. They generally pen down their taughts online. So our company want to help these people someone. And first problem that we have to we do not have model which can predict this wethere the tweets are related to suicide or not.</p>

## 🧿 **Objective**
 <p style="text-align: justify;">The objective of this project is that I will have to build a module which will try to classify the category of tweets. For this my company has given me a dataset which is in csv format. It has two columns, tweet and the suicide. Suicide column has labels whether the tweets are suicidical or not. So I will be using these data in the csv file and building an NLP model. Firstly I will be going through the full csv dataset and trying to remove the unwanted data. And then using these data I will be building a model. I will also be doing several experiments in order to improve the performance of the model. I will check if our dataset is balanced and if there is a requirement to increase the dataset so that our model can learn better from the data. Then I will choose the best-performing model to build my final model with the best-suited parameters.
 </p>

**About the data**  
This dataset contains 2 columns. This data set is from Kaggle. You can [**click here**](https://www.kaggle.com/datasets/aunanya875/suicidal-tweet-detection-dataset) to view the dataset on kaggle.

<a id='tableofcontent'></a>
## **Table of Content**

##### ⬇ [**Importing Libraries**](#library)
##### 📊 [**Importing Dataset**](#dataset)
##### 🗺️ [**Exploring the dataset**](#exploration)
##### 💭 [**My taughts on the dataset**](#thoughts)
##### ⚙️ [**Pre-processing the data**](#processing)
##### 🔧 [**Feature Engineering**](#feature)
##### 🗂️ [**Selecting the best model for our dataset**](#selection)
##### ✅ [**Assessing which gives us the best results**](#assess)
##### 🎛️ [**Hyper-Parameter Tuning**](#tuning)
##### 🏁 [**Final Model**](#Final)
##### 💡 [**Conclusion**](#conclusion)

## ***Let's begin....***

<a id='library'></a>
## ⬇ <span style="color: #20479b; font-weight: bold;">Importing Libraries</span> <a href="#jason" style="float: right; text-decoration: none; font-size: 24px;">⬆️</a>

I start by importing the necessary libraries for my Exploratory Data Analysis tasks in Python.

In [ ]:
import numpy as np
import  pandas as  pd
import re
import nltk
import seaborn as sns
import os
import warnings
import string
import emoji

from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.model_selection
from  imblearn.over_sampling import SMOTE
from  imblearn.over_sampling import ADASYN
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report,confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import geometric_mean_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import optuna
 

<a id='dataset'></a>
## 📊 <span style="color: #20479b; font-weight: bold;">Importing Dataset</span>

In [ ]:
dtst_dir = '/Users/jasonjoelpinto/Documents/GitHub/python-datascience-projects/016. sentiment_analysis_NLP_project/dataset/Suicide_Ideation_Dataset(Twitter-based).csv'
dtst = pd.read_csv(dtst_dir,nrows=10000)
print(display(dtst.sample(5)))

<a id='exploration'></a>
## 🗺️ <span style="color: #20479b; font-weight: bold;">Exploring the dataset</span> <a href="#toc" style="float: right; text-decoration: none; font-size: 24px;">⬆️</a>

<a id='exploration'></a>
## 🗺️ <span style="color: #20479b; font-weight: bold;">Exploring the dataset</span> <a href="#toc" style="float: right; text-decoration: none; font-size: 24px;">⬆️</a>

In [ ]:
dtst.sample(5)

I can see that 
1. ID column is not necessary for us, hence we can delete it.
2. tweets have alot of unnessary special characters, which we have to clean.

In [ ]:
dtst.shape

There are 1787 rows of data which is good so that the model can learn better from the dataset.

In [ ]:
dtst.info()

In [ ]:
dtst.dtypes

In [ ]:
null_percentage = {}
for column in dtst:
    null_percentage[column] = round(( ( dtst[column].isnull().sum() ) / len(dtst)) * 100,2)
np_dataset = pd.DataFrame( list( null_percentage.items() ),  columns=[  'column_name'  , 'null_percentage' ])
np_dataset

There are null entries in our dataset in tweet column.

In [ ]:
dtst["Suicide"].unique()

There are no unnessary entries too. Which is a good thing for us.

t_ct = dtst['Suicide'].value_counts()
print('Each class count:')
sns.barplot(x=t_ct.index, y=t_ct.values)
t_ct

Here we can see that our dataset is ** imbalanced**. We need to balance the dataset so that we can have good model trained.

<a id='thoughts'></a>
## 💭 <span style="color: #20479b; font-weight: bold;">My thoughts on the dataset</span>

As per my initial Data Exploration,
- There are null values.
- I can see that the dataset big enough to build a good ML model.
- I need to rename the suicide column to 0 and 1 
- I will be also restructuring the column and moving the labels to the end to ease of reading.
- The dataset consists of 31962 rows of data.
- There are usertags and special characters in the column tweet. We need to remove those.
- The dataset is heavily imbalanced. Hence we need to oversample the data.


Evaluation Metrics:
For this dataset I will be using, Accuracy, precision, F1 are the evaluation metrics.

Considering all these above points, I will be following the below steps to 

**⚙️ pre-process**
1. 🗑️ Delete the column ID.
2. 🫸🏻 Move the column label to the end.
3. 🔄 Convert the body to lower case
4. 🧹 Remove usertags.
5. 🧹 Remove special characters.
<!-- 4. ➗ Spliting our main datset into two parts, one part for training and another for testing dataframe
    - Doing it before testing or 
    - to avoid data Scalling and oversampling. -->

**🔧 feature Engineer**
<!-- 1. ⚖️ Scalling the numerical and categorical labels -->

<!-- 2. 📈 Oversampling the data for stroke = 1, Because of two main reasons.
    - Our focus is on stroke detection and data for people who have stroke is very less.
    - We do not have to worry about data size as well here, since it's not very huge. -->

<a id='processing' ></a>
## ⚙️ <span style="color: #20479b; font-weight: bold;">Pre-processing the data</span>


**1. 🗑️ Delete the null values**

In [ ]:
dtst = dtst.dropna()
dtst.head(5)

**2. Change the column values**

In [ ]:
dtst['Suicide'] = dtst['Suicide'].str.lower()
dtst['Suicide'] = dtst['Suicide'].str.replace(' ', '', regex=False)
def chg_col_vl(vl):
    if vl == 'notsuicidepost':
        return 0
    elif vl == 'potentialsuicidepost':
        return 1
    
dtst['Suicide'] =  dtst['Suicide'].apply(chg_col_vl)
dtst.head(5)

**3. 🔄 Convert the body to lower case**

In [ ]:
dtst['tweet_processed'] = dtst['Tweet'].str.lower()
dtst.head(5)

**4. #️⃣ Remove hashtags**

In [ ]:
dtst['tweet_processed'] = dtst[
                             'tweet_processed'
                             ].replace(
                                        r'#([^\s]+)', r'\1', regex=True
                                        )
dtst.head(5)

**5.  🧹 Remove usertags**

In [ ]:
dtst['tweet_processed'] = dtst['tweet_processed'].replace(r'@', '', regex=True)
dtst.head(5)  

**6.🗑️ Remove Punctuation**

In [ ]:
def rem_punc(tweet):
    tweet = tweet.translate(str.maketrans('','',string.punctuation))
    return tweet

dtst['tweet_processed'] = dtst['tweet_processed'].apply(rem_punc)
dtst.head(5)

**7. 🔢 Clear all numbers**

In [ ]:
dtst['tweet_processed'] = dtst[
                            'tweet_processed'
                            ].replace(
                                        r'\d+', '', regex=True
                                        )
dtst.head(5)

**8. 🧹 Remove special characters**

In [ ]:
dtst['tweet_processed'] = dtst['tweet_processed'].str.replace("[^a-zA-Z#]", " ")
dtst.head()

**9. Remove unwanted giberish characters**

In [ ]:
def rem_gib_space(twt):
    twt = twt.encode('ascii', 'ignore').decode('ascii')
    twt = re.sub(r'\s+', ' ', twt).strip()
    return twt

dtst['tweet_processed'] = dtst['tweet_processed'].apply(rem_gib_space)
dtst.head(10)

In [ ]:
dtst.head(5)

In [ ]:
# def preprocessor(text):
#     text = word_tokenize(text.lower())
#     text = [PorterStemmer().stem(word) for word in text if not word in set(stopwords.words('english'))]
#     text = ' '.join(text)
#     return text

# dtst['tweet_processed'] = dtst['tweet_processed'].apply(preprocessor)

dtst.head()

In [ ]:

# porter = nltk.SnowballStemmer('english')

# def preprocessor(input_text):
#     tokenized_text = nltk.word_tokenize(input_text, language='english')
#     stems = [porter.stem(t) for t in tokenized_text]
#     return ' '.join(stems)

# dtst['tweet_processed'] = dtst['tweet_processed'].apply(preprocessor)

# dtst.head(10)

**10. Creating a TD-IDF matrix for the tweets**

In [ ]:
# dtst['tweet_processed'] = dtst['tweet_processed'].fillna('')

In [ ]:
trm_mdl = TfidfVectorizer(min_df=2)
m3 = trm_mdl.fit_transform(dtst['tweet_processed'])
inf = pd.DataFrame(m3.toarray(), index=dtst.index)
dtst = dtst.join(inf)
inverted_index = {id: term for term, id in trm_mdl.vocabulary_.items()}
dtst.head(5)

There are unwanted characters in my tweets data. How can I remove those? Here is some examples

**11. ❎ Splitting the data**

In [ ]:
tr = dtst['Suicide'].values
tr = tr.astype(float)

ft = dtst.drop(columns=[
                        'Tweet',
                        'tweet_processed',
                        'Suicide'
                        ])
ft
# ft = vectorizer.fit_transform(dtst['tweet_processed']).toarray()
# ft

In [ ]:
print(f'No of features     : {ft.shape[1]}')
print(f'Total rows of data : {ft.shape[0]}')

In [ ]:
# ft_lst = []
# for itm in ft.columns:
#     if itm in inverted_index:
#         ft_lst.append(inverted_index[itm])
#     else:
#         ft_lst.append(itm)

# ft = ft.values
# ft = ft.astype('float')
# ft_lst

In [ ]:
ft_train_df,  ft_val_test_df,   tr_train_df,  tar_val_test_df = sklearn.model_selection.train_test_split(  ft , tr  ,  test_size = 0.35, shuffle=True, stratify=tr, random_state  =  40)
ft_val_df,  ft_test_df,   tar_val_df,  tar_test_df = sklearn.model_selection.train_test_split(  ft_val_test_df , tar_val_test_df  ,  test_size = 0.50,shuffle=True, stratify=tar_val_test_df,random_state  =  40)

print(f'Train features shape     : {ft_train_df.shape}')
print(f'Train target shape       : {tr_train_df.shape}')
print(39*'=')
print(f'Validation features shape: {ft_val_df.shape}')
print(f'Validation target shape  : {tar_val_df.shape}')
print(39*'=')
print(f'Test features shape      : {ft_test_df.shape}')
print(f'Test target shape        : {tar_test_df.shape}')
print(39*'=')


<a id='feature' ></a>
## 🔧 <span style="color: #20479b; font-weight: bold;">Feature Engineering</span>

**1. 📈 Oversampling the data**

Applying **SMOTE** oversampling to the dataset

In [ ]:
mod_smt =  SMOTE(  )
ft_train_df_smt, tr_train_df_smt   =    mod_smt.fit_resample(
                                                            ft_train_df, 
                                                            tr_train_df)

print("fet_train_smot size :",ft_train_df_smt.shape)
print("tar_train_smot size :",tr_train_df_smt.shape)

Applying **ADASYN** oversampling to the dataset

In [ ]:
mod_adc =  ADASYN(  )
ft_train_df_adc, tr_train_df_adc   =    mod_adc.fit_resample(
                                                            ft_train_df, 
                                                            tr_train_df)

print("fet_train_smot size :",ft_train_df_adc.shape)
print("tar_train_smot size :",tr_train_df_adc.shape)

<a id='selection'></a>
## 🗂️ <span style="color: #20479b; font-weight: bold;">Selecting the best model</span>

In [ ]:
log_reg = sklearn.linear_model.LogisticRegression()
svc = sklearn.svm.SVC()
ran_for_cls =  sklearn.ensemble.RandomForestClassifier()
xg_bst = XGBClassifier()
# k_n_n =   sklearn.neighbors.KNeighborsClassifier()
# gb = sklearn.ensemble.GradientBoostingClassifier()

In [ ]:
def evaluate_model(ml, trf, trl, vlf, vll, mtd):
        ml.fit(trf, trl)
        p = pd.Series(ml.predict(vlf))
        return [
            mtd,
            ml.__class__.__name__,
            geometric_mean_score(vll, p),
            f1_score(vll, p),
            precision_score(vll, p),
            recall_score(vll, p),
            accuracy_score(vll, p)
                ]

mt_vl_lts = []
lt = [log_reg,svc,ran_for_cls,xg_bst]
cmb = [
        ('IM', ft_train_df, tr_train_df),
        ('SM', ft_train_df_smt, tr_train_df_smt),
        ('AD', ft_train_df_adc, tr_train_df_adc)
        ]
for ml in lt:
    for mtd, trf, trl in cmb:
        mt_vl_lt = evaluate_model(ml, trf, trl, ft_val_df, tar_val_df, mtd)
        if mt_vl_lt:
            mt_vl_lts.append(mt_vl_lt)
mt_vl_lts = pd.DataFrame(mt_vl_lts, columns=['Method', 'Model Name', 'G-mean','F1','PRE','REC', 'ACC'])
mt_vl_lts = mt_vl_lts.sort_values(ascending=False, by=['G-mean','F1','PRE','REC', 'ACC'])

display(mt_vl_lts.head(30))

print('\n')
print('*****'*15)
bm_name = mt_vl_lts.head(1)['Model Name'].values[0]
osty = mt_vl_lts.head(1)['Method'].values[0]
print(f'The best performing model is  :{bm_name} with oversampling method : {osty}')
print('*****'*15)

model_mapping = {
    'LogisticRegression': LogisticRegression,
    'SVC': SVC,
    'RandomForestClassifier': RandomForestClassifier,
    'XGBClassifier': XGBClassifier
}

# Retrieve the best model class
bm = model_mapping[bm_name]

In [ ]:
tuning_dsets = {
    'AD': {'X': ft_train_df_adc, 'y': tr_train_df_adc},
    'SM': {'X': ft_train_df_smt, 'y': tr_train_df_smt},
    'IM': {'X': ft_train_df, 'y': tr_train_df},
}
ft = tuning_dsets[osty]['X']
tr = tuning_dsets[osty]['y']

<a id='Hyper'></a>
## 🎛️ <span style="color: #20479b; font-weight: bold;">Hyperparameter Tuning</span>

In [ ]:
def hyptune(t):
    prs = {
        'objective': 'binary:logistic',
        'n_estimators': trial.suggest_int('n_estimators', 80, 950),
        'max_depth': trial.suggest_int('max_depth', 2, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'booster': 'gbtree',
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0)
            }

    hyml = bm(**prs, use_label_encoder=False)
    hyml.fit(ft, tr)
    hyp = hyml.predict(ft_val_df)
    gm  = geometric_mean_score(tar_val_df, hyp)
    fs  = f1_score(tar_val_df, hyp)
    pre = precision_score(tar_val_df, hyp)
    rec = recall_score(tar_val_df, hyp)
    return gm
    
optuna.logging.set_verbosity(optuna.logging.INFO)
lrn = optuna.create_study(direction='maximize')
lrn.optimize(objective, n_trials=200)
bt = lrn.best_trial.value
bp = lrn.best_trial.params
print(f'Top gmean score          : {bt}')
print(f'Top performing parameters: {bp}')

<a id='Final'></a>
## 🏁 <span style="color: #20479b; font-weight: bold;">Final Model</span>

In [ ]:
display(print(bp))
display(print(bm))

In [ ]:
bp = study.best_trial.params
fim = bm(**bp, use_label_encoder=False)
fim.fit(ft, tr)
fip = fim.predict(ft_test_df)
fgm  = geometric_mean_score(tar_test_df, fip)
ffs  = f1_score(tar_test_df, fip)
fpre = precision_score(tar_test_df, fip)
frec = recall_score(tar_test_df, fip)

print(f'G-Mean Score: {fgm}')
print(f'F1 Score    : {ffs}')
print(f'Precision   : {fpre}')
print(f'Recall      : {frec}')

<a id='Evaluation'></a>
## 💡 <span style="color: #20479b; font-weight: bold;"> Model Evaluation</span>

Classification report

In [ ]:
print(classification_report(tar_test_df, fip))

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(tar_test_df, fip)).plot()

As we can see in the above plot, we were able to get very good scored and out of the 6 images we were able to predict most of it.

<a id='conclusion'></a>
## 💡 <span style="color: #20479b; font-weight: bold;">Conclusion</span>

<a target="_blank" href="https://icons8.com/icon/68193/twitter">Twitter</a> icon by <a target="_blank" href="https://icons8.com">Icons8</a>